# Philips Digital Diagnost (C90) - Image Data Table Parser

This notebook takes the output of the QA Tool > Image Data Table export from a Philips Digital Diagnost and produces a .csv file that can be appended to the Reject Analysis and Dose Metric Dashboard database.

Let's start by importing the required libraries needed to make the parser work.

In [2]:
import pandas as pd
import os

from ipywidgets import *
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

The template for the database used in the Reject Analysis and Dose Metric Dashboard is as follows:

In [3]:
cols_list_final = ['Asset Number','Manufacturer','Model','Image Date','Image Time','Body Part','View','Exposure Index','KAP (uGy.m2)','kVp','Exposure (mAs)','Exposure time (ms)','Image Status','Reject Reason']

df_template = pd.DataFrame(columns=[*cols_list_final])
df_template

,Asset Number,Manufacturer,Model,Image Date,Image Time,Body Part,View,Exposure Index,KAP (uGy.m2),kVp,Exposure (mAs),Exposure time (ms),Image Status,Reject Reason


## Selecting file from file browser

To clean up an export, you'll need to select the exported files from the Philips QA Tool using the "File Select" button below.
<br>
You will also be asked for your asset number so that we can group systems in the Dashboard by facility.

In [4]:
# Select input Image Data Table file as exported from Philips QA tool

def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename() # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.

fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

## Adding Asset Number

In the next section, the file that you've selected will be printed out and you'll be asked to input the Asset Number of the system. Type the Asset Number in the box provided and then press "ENTER".

In [4]:
files = fileselect.files
print("The file you've selected is: ", files)
print()
print("What is the asset number of the system?")

AssetNumber = input()


The file you've selected is:  C:/Users/bernardm/Desktop/Test/Copy of Philips DigiDiagnost C90 example log.xlsx

What is the asset number of the system?
123456789


## Clean up dataframe

Let's do some clean-up on the .csv file:
- Define the separator. By default, this will be ";". However, this can be set to something different during export (| or , or -). Change the code below if your separator is something other than ";".

In [21]:
df = pd.read_excel(files)
df

,Image date,Image time,Study description,Protocol step name step,Unnamed: 4,kVp,Exposure [mAs],Exposure time [ms],Exposure Index (EI_s),Target Index (EI_T),Deviation Index (DI),Image dose area product [ÂµGy mÂ²],Operators name,Status,Reject reason,Modality,Image link
0,2020-08-11,18:41:11,DR Long Leg both,AP AEC,NaN,85.0,1.0,1.0,266.0,250.0,0.270,1.65,Service,rejected,Service testing,DX,[Image]
1,2020-08-11,18:41:12,DR Long Leg both,AP AEC,NaN,77.0,1.0,7.0,90.0,250.0,-4.454,0.68,Service,rejected,Service testing,DX,[Image]
2,2020-08-11,18:41:14,DR Long Leg both,AP AEC,NaN,70.0,1.0,11.0,73.0,250.0,-5.330,0.33,Service,rejected,Service testing,DX,[Image]
3,2020-08-11,18:41:15,DR Long Leg both,AP AEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Service,rejected,Service testing,DX,[Image]
4,2020-08-12,15:12:37,Unknown RIS Code,Unknown RIS Code,NaN,96.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-08-13,12:31:57,Emergency Examination,Emergency View1,NaN,77.0,2.0,9.0,255.0,250.0,0.093,0.98,eleva,confirmed,NaN,DX,[Image]
6,2020-08-13,13:04:09,Emergency Examination,Emergency View2,NaN,73.0,6.0,10.0,538.0,250.0,3.331,12.18,eleva,confirmed,NaN,DX,[Image]
7,2020-08-13,13:02:23,Emergency Examination,Emergency View2,NaN,73.0,6.0,10.0,492.0,250.0,2.938,2.30,eleva,confirmed,NaN,DX,[Image]
8,2020-08-13,12:45:28,Emergency Examination,Emergency View2,NaN,73.0,50.0,51.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-08-13,12:45:13,Emergency Examination,Emergency View2,NaN,73.0,40.0,41.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Adding unique identifiers

If the .csv file has been read in correctly above, you should see a table with all the values from the log. 
<br>
Let's add a few identifiers for filtering purposes:
- Asset Number
- Manufacturer
- Model


In [22]:
cols_list = ['Asset Number','Manufacturer','Model']
df = df.reindex(columns=[*cols_list,*df.columns.tolist()])

df['Asset Number'] = AssetNumber
df['Manufacturer'] = "Philips"
df['Model'] = "Digital Diagnost C90"
df

,Asset Number,Manufacturer,Model,Image date,Image time,Study description,Protocol step name step,Unnamed: 4,kVp,Exposure [mAs],Exposure time [ms],Exposure Index (EI_s),Target Index (EI_T),Deviation Index (DI),Image dose area product [ÂµGy mÂ²],Operators name,Status,Reject reason,Modality,Image link
0,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:11,DR Long Leg both,AP AEC,NaN,85.0,1.0,1.0,266.0,250.0,0.270,1.65,Service,rejected,Service testing,DX,[Image]
1,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:12,DR Long Leg both,AP AEC,NaN,77.0,1.0,7.0,90.0,250.0,-4.454,0.68,Service,rejected,Service testing,DX,[Image]
2,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:14,DR Long Leg both,AP AEC,NaN,70.0,1.0,11.0,73.0,250.0,-5.330,0.33,Service,rejected,Service testing,DX,[Image]
3,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:15,DR Long Leg both,AP AEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Service,rejected,Service testing,DX,[Image]
4,123456789,Philips,Digital Diagnost C90,2020-08-12,15:12:37,Unknown RIS Code,Unknown RIS Code,NaN,96.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,123456789,Philips,Digital Diagnost C90,2020-08-13,12:31:57,Emergency Examination,Emergency View1,NaN,77.0,2.0,9.0,255.0,250.0,0.093,0.98,eleva,confirmed,NaN,DX,[Image]
6,123456789,Philips,Digital Diagnost C90,2020-08-13,13:04:09,Emergency Examination,Emergency View2,NaN,73.0,6.0,10.0,538.0,250.0,3.331,12.18,eleva,confirmed,NaN,DX,[Image]
7,123456789,Philips,Digital Diagnost C90,2020-08-13,13:02:23,Emergency Examination,Emergency View2,NaN,73.0,6.0,10.0,492.0,250.0,2.938,2.30,eleva,confirmed,NaN,DX,[Image]
8,123456789,Philips,Digital Diagnost C90,2020-08-13,12:45:28,Emergency Examination,Emergency View2,NaN,73.0,50.0,51.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,123456789,Philips,Digital Diagnost C90,2020-08-13,12:45:13,Emergency Examination,Emergency View2,NaN,73.0,40.0,41.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Putting data into the template dataframe

Let's match the .csv columns into the template:
- rename the columns of the original .csv file to match the template
- remove any columns we don't need
- rearrange the columns to match the template

In [24]:
df = df.rename(columns={"Study description": "Body Part",
                       "Protocol step name step": "View",
                        "Exposure Index (EI_s)": "Exposure Index",
                        "Exposure [mAs]": "Exposure (mAs)",
                        "Exposure time [ms]": "Exposure time (ms)",
                        "Status": "Image Status",
                        "Reject reason": "Reject Reason",
                        "Image dose area product [ÂµGy mÂ²]": "KAP (uGy.m2)",
                        "Image date": "Image Date",
                        "Image time": 'Image Time'
                       })

df_out = df[[*cols_list_final]]
df_out

,Asset Number,Manufacturer,Model,Image Date,Image Time,Body Part,View,Exposure Index,KAP (uGy.m2),kVp,Exposure (mAs),Exposure time (ms),Image Status,Reject Reason
0,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:11,DR Long Leg both,AP AEC,266.0,1.65,85.0,1.0,1.0,rejected,Service testing
1,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:12,DR Long Leg both,AP AEC,90.0,0.68,77.0,1.0,7.0,rejected,Service testing
2,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:14,DR Long Leg both,AP AEC,73.0,0.33,70.0,1.0,11.0,rejected,Service testing
3,123456789,Philips,Digital Diagnost C90,2020-08-11,18:41:15,DR Long Leg both,AP AEC,NaN,NaN,NaN,NaN,NaN,rejected,Service testing
4,123456789,Philips,Digital Diagnost C90,2020-08-12,15:12:37,Unknown RIS Code,Unknown RIS Code,1.0,NaN,96.0,1.0,1.0,NaN,NaN
5,123456789,Philips,Digital Diagnost C90,2020-08-13,12:31:57,Emergency Examination,Emergency View1,255.0,0.98,77.0,2.0,9.0,confirmed,NaN
6,123456789,Philips,Digital Diagnost C90,2020-08-13,13:04:09,Emergency Examination,Emergency View2,538.0,12.18,73.0,6.0,10.0,confirmed,NaN
7,123456789,Philips,Digital Diagnost C90,2020-08-13,13:02:23,Emergency Examination,Emergency View2,492.0,2.30,73.0,6.0,10.0,confirmed,NaN
8,123456789,Philips,Digital Diagnost C90,2020-08-13,12:45:28,Emergency Examination,Emergency View2,3.0,NaN,73.0,50.0,51.0,NaN,NaN
9,123456789,Philips,Digital Diagnost C90,2020-08-13,12:45:13,Emergency Examination,Emergency View2,3.0,NaN,73.0,40.0,41.0,NaN,NaN


## Output of dataframe into .csv file

Check the output table above and confirm that the information matches the column names. 
If everything is OK, let's export the cleaned up .csv file into an output file. 

A .csv file will be created with the Asset Number and a datestamp.

In [25]:
# Add timestamp to filename
from datetime import datetime
date = datetime.now().strftime("%Y_%m_%d_%I_%S%p")

df_out.to_csv((r'C:\Users\BernardM\GitHub\JupyterNotebooks\rejectAnalysis\outputdata\\'
               +str(AssetNumber)
               +'_'
               +str(date)
               +'.csv')
               ,index = False, header = True)

print("The output file has been successfully created.")

The output file has been successfully created.


This output file can now be appended to the Reject Analysis and Dose Metric Dashboard database.